In [1]:
!pip3 install -r requirements.txt
!pip3 install datasets trl peft bitsandbytes wandb accelerate

     |████████████████████████████████| 12.8 MB 48 kB/s  eta 0:00:01
     |████████████████████████████████| 547 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 33.9 MB/s eta 0:00:01
     |████████████████████████████████| 251 kB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 119.8 MB 4.9 MB/s eta 0:00:011.4 MB/s eta 0:00:02     |███████████████████             | 71.1 MB 1.2 MB/s eta 0:00:42     |████████████████████▏           | 75.4 MB 1.2 MB/s eta 0:00:39     |████████████████████▋           | 77.1 MB 1.2 MB/s eta 0:00:37███████████████▋          | 80.8 MB 1.2 MB/s eta 0:00:34
     |████████████████████████████████| 6.9 MB 23.0 MB/s eta 0:00:01     |████████████████████████▊       | 5.3 MB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 6.4 MB/s eta 0:00:01
  Using cached pyarrow-16.1.0-cp39-cp39-manylinux_2_28_x86_64.whl (40.8 MB)
  Using cached multiprocess-0.70.16-py39-none-any.whl (133 kB)
     |█████████████████████████

In [2]:
from haystack import Pipeline
from haystack.components.preprocessors import TextCleaner, DocumentCleaner, DocumentSplitter
from haystack.components.converters import HTMLToDocument
from huggingface_hub import notebook_login

In [3]:
notebook_login()

In [9]:
# pipeline = Pipeline()
# pipeline.add_component("cleaner", TextCleaner(remove_punctuation=True, convert_to_lowercase=True))
# pipeline.run({"cleaner":{"texts":["Hello World$"]}})
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator
from transformers import BitsAndBytesConfig
import torch

prompt_template = """
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
	You are a privacy policy analyzer. Your job is to evaluate the given set of documents and list
	any personally identifiable information that is collected by the third party service, which
	includes details about the user themselves (such as their name and address), or information
	about their device(s) (such as their geographic location, device ID, advertising ID, device
 	settings or configurations, network details such as WiFi SSIDs, Interet Protocol Addresses,
  	MAC Addresses) This detail can also include information collected automatically by the service
   provider about user interactions, such as image and audio information, any type of metadata,
   cookies, and tracking across other websites and applications.

   Analyze these documents with this criteria:
   {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    <|start_header_id|>assistant<|end_header_id|>
"""

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# generator = HuggingFaceLocalGenerator(
#     model="meta-llama/Meta-Llama-3-8B-Instruct",
#     huggingface_pipeline_kwargs={"device_map":"auto",
#                                   "model_kwargs":{"quantization_config": bnb_config}},
#     generation_kwargs={"max_new_tokens": 500})

pipeline = Pipeline()
pipeline.add_component("converter", HTMLToDocument())
pipeline.add_component("cleaner", DocumentCleaner(remove_regex='[^A-Za-z0-9 !.:/]+',
   remove_empty_lines=True, remove_repeated_substrings=True))
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=5, split_overlap=0))
pipeline.add_component("prompt_builder", PromptBuilder(template=prompt_template))
# pipeline.add_component("llm", generator)

pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter.documents", "prompt_builder.documents")
# pipeline.connect("prompt_builder", "llm")


🚅 Components
  - converter: HTMLToDocument
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - prompt_builder: PromptBuilder
🛤️ Connections
  - converter.documents -> cleaner.documents (List[Document])
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> prompt_builder.documents (List[Document])

In [10]:
results = pipeline.run({"converter": {"sources": ["tiktok_policy.html"]}})

In [11]:
print(results['prompt_builder']['prompt'])


    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
	You are a privacy policy analyzer. Your job is to evaluate the given set of documents and list
	any personally identifiable information that is collected by the third party service, which
	includes details about the user themselves (such as their name and address), or information
	about their device(s) (such as their geographic location, device ID, advertising ID, device
 	settings or configurations, network details such as WiFi SSIDs, Interet Protocol Addresses,
  	MAC Addresses) This detail can also include information collected automatically by the service
   provider about user interactions, such as image and audio information, any type of metadata,
   cookies, and tracking across other websites and applications.

   Analyze these documents with this criteria:
   
        What Information We CollectWe may collect information from and about you including information that you provide information from other sources and au